In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('/Users/theot/Documents/SEGR_0D/EGR0D/'))
if module_path not in sys.path:
    sys.path.append(module_path)

#sys.path.remove('/Users/theot/Documents/SEGR_0D/EGR0D/')
print(sys.path)

['/Users/theot/Documents/SEGR_0D/EGR0D/src', '/Users/theot/Documents/SEGR_0D/EGR0D/src', '/Users/theot/Documents/arcane/ARCANE', '/opt/homebrew/anaconda3/envs/can260/lib/python39.zip', '/opt/homebrew/anaconda3/envs/can260/lib/python3.9', '/opt/homebrew/anaconda3/envs/can260/lib/python3.9/lib-dynload', '', '/opt/homebrew/anaconda3/envs/can260/lib/python3.9/site-packages', '/Users/theot/Documents/SEGR_0D/EGR0D']


In [2]:
from new_egr import *
import time
#from egr_multiproc import *
import pandas as pd

In [ ]:
if __name__ == '__main__':
    # get the start time
    st = time.time()

    # create a dictionary to store the simulation parameters, containing, fuel, oxidizer and dilutent composition, temeratures of this components, pressure, equivalence ratio, EGR rate with its units, and the simulation type, and kinetic mechanism
    param = {'fuel': {'compo':'CH4:1.','T':fill_T_param(),'P':[100000],'scheme':'Aramco13.cti'},
             'oxidizer': {'compo':'O2:1. N2:3.76','T':[300],'P':[100000],'scheme':'air.xml'},
             'dilutent': {'compo':'CO2:0.5','T':[300],'P':[100000],'scheme':'Aramco13.cti'},
             'fuel_temp': [300.0],
             'oxidizer_temp': [300.0],
             'dilutent_temp': [800.0],
             'fuel_pressure': [100000],
             'oxidizer_pressure': [100000],
             'dilutent_pressure': [100000],
             'phi_range': np.arange(0.7,1.35,0.05),
             'egr_range': np.arange(0.0,0.11,0.05),
             'egr_rate_unit': 'mol',
             'scheme': 'Aramco13.cti',
             'sim_type': '0D'}

    def fill_T_param(param, T):
        for T in param['fuel_temp']:
            yield T
    # create a dictionary to store the simulation results
    results = {'phi': [],
                'T': [],
                'P': [],
                'EGR': [],
                'X': [],
                'Y': [],
                'u': [],
                'HRR': [],
                'grid': []}



    config = case('CH4:1.',                   #fuel compo
                'O2:1. N2:3.76',              #ox compo
                'CO2:0.5',                    #egr compo
                3960.0,                       #thermal output power (kind of useless)
                0.0001,                       #egr rate
                'mol'                         #egr rate unit
                )
    dfs=[]
    pressures = [100000,500000] #initial pressures to test in Pa
    for p in pressures:
        #set reservoirs thermo-state
        config.res.fuel = create_reservoir(param['fuel'])
        config.res.ox = create_reservoir(param['oxidizer'])
        config.res.egr = create_reservoir(param['dilutent'])

        #range of computation
        egr_percentages = np.arange(0.0,0.11,0.05)
        df = pd.DataFrame()
        for egr_rate in egr_percentages:
            config.egr_rate = egr_rate #override config.egr_rate set during object instanciation
            phi_range = np.arange(0.7,1.35,0.05)
            reactor,results,pdresult = compute_solutions_1D(config,phi_range,power_regulation=False)
            #plt.plot(phi_range,results[:,4],label='EGR reacteurs:'+str(round(config.egr_rate*100,1)),marker='o')
            #subplot_data(phi_range,results,'Phi',['T[K]','HRR[W/m3]','Y_O2','Y_CO2'],'EGR rate (%):'+str(round(config.egr_rate*100,1))+'%')
            df=pd.concat([df,pdresult])
        
        #df = df.pivot_table(index='phi',columns='EGR',values='u')
        #print_reactor(df)
        # get the end time
        et = time.time()

        # get the execution time
        elapsed_time = et - st
        print('Execution time:', elapsed_time, 'seconds')
        dfs.append(df)

In [ ]:
dfsi=dfs
filenames = ['df'+str(idx)+'.pkl' for idx in range(len(dfsi))]
[df.to_pickle(filenames[idx]) for idx,df in enumerate(dfsi)]
params={'pressure':pressures,
        'filename':filenames,
        'scheme':'Aramco13.cti'}

In [ ]:
#filenames = ['df'+str(idx)+'.pkl' for idx in range(2)]

dfsr = []
[dfsr.append(pd.read_pickle(file)) for file in filenames]
dfs_unique_var=[df.pivot_table(index='phi',columns='EGR',values='T') for df in dfsr]

fig, ax = plt.subplots(1,1)

dfsc=pd.concat(dfs_unique_var, axis = 1)

human_labels = [(str(round(p/100000,1))+' bar, '+str(round(e*100,1))+"%EGRmol") for p in pressures for e in egr_percentages]
T=800
dfsc.plot(ax=ax, style='--o',title='(1D) Tad vs equivalence ratio'+'(Tin_EGR:'+str(round(T,0))+'K)',
            #Pin:'+str(round(P/100000,0))+'bar)',
            xlabel='Equivalence ratio',ylabel='T [K]')

#equilibrate_data=main()
#equilibrate_data=equilibrate_data.pivot_table(columns='EGR',index='phi',values='T')
#equilibrate_data.plot(ax=ax, style='--x',title='Temperature vs equivalence ratio',xlabel='Equivalence ratio',ylabel='T',legend=False, label='EGR Reactor')

plt.grid()
plt.legend(loc='best')
ax.legend(human_labels)
plt.show()